# Similarity Based Recommendation-System (item-item)

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 19 20:59:02 2019

@author: Abdulrahman Alothman
"""
#iki filmin arasındaki benzerliği hesaplanıyor .(based similarity) 

import numpy as np # linear algebra
import pandas as pd # data processing

rating = pd.read_csv('rating.csv',encoding = "latin-1")
anime = pd.read_csv('anime.csv')

In [4]:

anime = anime.astype({"name":'category',"genre":'category',"episodes":'category','type':'category' })
print(anime.isnull().sum().to_frame('nulls'))

#eksik veriler doldurulur
anime.rating=anime.rating.fillna(anime.rating.mean())
anime["genre"]=(anime["genre"].cat.codes.replace(-1, np.nan).interpolate().astype(int).astype('category').cat.rename_categories(anime["genre"].cat.categories))
anime["type"]=(anime["type"].cat.codes.replace(-1, np.nan).interpolate().astype(int).astype('category').cat.rename_categories(anime["type"].cat.categories))
print(anime.isnull().sum().to_frame('nulls'))

#-1 ler silme
df_rating1 = pd.merge(rating,anime.drop('rating',axis=1),on='anime_id')
df_rating=df_rating1[df_rating1.rating!=-1]
df_rating=df_rating.iloc[0:3000000,:]
df_rating.head(7)


          nulls
anime_id      0
name          0
genre         0
type          0
episodes      0
rating        0
members       0
          nulls
anime_id      0
name          0
genre         0
type          0
episodes      0
rating        0
members       0


,user_id,anime_id,rating,name,genre,type,episodes,members
1,3,20,8,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
2,5,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
5,21,20,8,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
6,28,20,9,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
7,34,20,9,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
8,38,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
9,39,20,10,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297


In [5]:
pivot_table = df_rating.pivot_table(index = ["user_id"],columns = ["anime_id"],values = "rating")
pivot_table.head(10)

anime_id,6,15,17,18,20,22,24,30,31,32,...,31043,31240,31338,31636,31722,31845,31859,31964,32182,32828
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN
5,8.0,6.0,6.0,6.0,6.0,5.0,1.0,1.0,NaN,NaN,...,NaN,8.0,NaN,NaN,7.0,NaN,NaN,NaN,9.0,7.0
7,NaN,NaN,NaN,NaN,NaN,7.0,NaN,10.0,9.0,9.0,...,9.0,NaN,6.0,NaN,NaN,5.0,7.0,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#checks the corrrelations between other movies on the table and Bad Boys.(ilgili filimle diğer filmler arsaındaki benzerlik)

movie_watched = pivot_table[225]  # 225 id'li film Dragon Ball GT
similarity_with_other_movies = pivot_table.corrwith(movie_watched)  # izlenen filmle diğer filmler arasındaki bezerliği bul
similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
similarity_with_other_movies.head(8)

anime_id
6695     1.000000
225      1.000000
987      0.681192
13561    0.605201
21781    0.599513
4667     0.596089
2907     0.572503
23301    0.567252
dtype: float64

In [10]:
similarity_movies_DF=pd.DataFrame(similarity_with_other_movies)
similarity_movies_scores=similarity_movies_DF.iloc[0:50,:]
anime_idd=similarity_movies_scores.index

recommended=df_rating.loc[df_rating.anime_id.isin(anime_idd)]
recommended=recommended.drop_duplicates(subset=['anime_id'])

#Dragon Ball GT dizisine benzeyen diziler
recommended.head(10)

,user_id,anime_id,rating,name,genre,type,episodes,members
114692,5,936,5,Naruto Movie 2: Dai Gekitotsu! Maboroshi no Ch...,"Adventure, Comedy, Drama, Fantasy, Shounen, Su...",Movie,1,97308
159517,5,2144,1,Naruto Movie 3: Dai Koufun! Mikazuki Jima no A...,"Action, Adventure",Movie,1,91526
275190,5,5667,2,To LOVE-Ru OVA,"Comedy, Ecchi, Harem, School, Sci-Fi, Shounen",OVA,6,79923
705120,531,13561,7,Guilty Crown: 4-koma Gekijou,Comedy,Special,11,13053
1142530,3,225,9,Dragon Ball GT,"Action, Adventure, Comedy, Fantasy, Magic, Sci...",TV,64,226625
1185034,3,813,10,Dragon Ball Z,"Action, Adventure, Comedy, Fantasy, Martial Ar...",TV,291,375662
1205710,3,1121,7,Pokemon Advanced Generation: Nanayo no Negaibo...,"Adventure, Comedy, Drama, Fantasy, Kids",Movie,1,43444
1208381,3,1122,7,Pokemon Advanced Generation: Rekkuu no Houmons...,"Action, Adventure, Comedy, Fantasy, Kids, Sci-Fi",Movie,1,45203
1211091,3,1132,8,Digimon Frontier,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",TV,50,74929
1270187,3,1564,7,Pokemon Advanced Generation,"Action, Adventure, Comedy, Fantasy, Kids",TV,192,67386
